In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [3]:
ip_df = pd.read_csv('IpAddress_to_Country.csv')

In [4]:
fraud_df = pd.read_csv('Fraud_Data.csv')

In [5]:
ip_df['upper_lower'] = list(zip(ip_df.loc[:,'lower_bound_ip_address'], ip_df.loc[:, 'upper_bound_ip_address']))

In [6]:
dict_df = ip_df.loc[:, ['country', 'upper_lower']]

Map IP addresses to the corresponding country in a dictionary

In [7]:
d = dict_df.groupby('upper_lower')['country'].apply(list).to_dict()


#### Create a function to extract countries from ip addresses using ip address dictionary

In [8]:
upper_bounds=[(v[0], k[1]) for k,v in d.items()]
def get_country(x):
    for c,ip in upper_bounds:
        if x<=ip: return c  # didn't end up using

### Feature Engineering

#### Find the countries for all the IP addresses listed

In [9]:
v = ip_df.loc[:, 'lower_bound_ip_address':'upper_bound_ip_address'].apply(tuple, 1).tolist()
idx = pd.IntervalIndex.from_tuples(v, closed='both')
fraud_df.loc[:, 'country'] = ip_df.loc[idx.get_indexer(fraud_df['ip_address'].values), 'country'].values

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1027: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


#### Make dummies for the source and browser

In [37]:
fraud_df = fraud_df.merge(pd.get_dummies(fraud_df.source), left_index=True, right_index=True)

In [38]:
fraud_df = fraud_df.merge(pd.get_dummies(fraud_df.browser), left_index=True, right_index=True)

In [39]:
fraud_df['sex'] = pd.Series(np.where(fraud_df.sex.values == 'F', 1, 0),
          fraud_df.index)

In [40]:
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'])

In [41]:
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'])

#### Find difference in signup time and purchase time

In [42]:
fraud_df['time_to_purchase'] = fraud_df['purchase_time'] -fraud_df['signup_time']

In [43]:
fraud_df.time_to_purchase = fraud_df.time_to_purchase.apply(lambda x: x / timedelta (days=1))

split dataframe and find percent countries in test set above a certain threshold

In [44]:
date_to_split = dt.datetime(2015, 7, 31) #split on this date for the train/test split

In [45]:
calculate_percent_fraud = fraud_df[fraud_df.signup_time <= date_to_split]

In [46]:
percent_fraud_df = pd.DataFrame(calculate_percent_fraud.groupby(['country'])['class'].sum()/fraud_df.groupby(['country'])['class'].count())

#### Find countries with fraud percent above 20

In [47]:
fraud_countries = list(percent_fraud_df.loc[percent_fraud_df['class'] > 0.2].index)

In [48]:
def possible_fraud(country):
    if country in fraud_countries:
        return 1
    else:
        return 0

In [49]:
fraud_df['possible_fraud'] = fraud_df['country'].apply(lambda x: possible_fraud(x))

In [50]:
fraud_df.isna().sum()  #see if there are any missing values

user_id                 0
signup_time             0
purchase_time           0
purchase_value          0
device_id               0
source                  0
browser                 0
sex                     0
age                     0
ip_address              0
class                   0
country             21966
time_to_purchase        0
possible_fraud          0
Ads                     0
Direct                  0
SEO                     0
Chrome                  0
FireFox                 0
IE                      0
Opera                   0
Safari                  0
dtype: int64

#### Create X and y matrices for model

In [51]:
X = fraud_df.loc[:, ['purchase_value', 'sex', 'age', 'Ads', 'Direct', 'SEO', 'time_to_purchase', 'Chrome',
       'FireFox', 'IE', 'Opera', 'Safari', 'possible_fraud']]

In [52]:
y = fraud_df.loc[:, ['class']]

In [31]:
df_train = fraud_df[fraud_df.signup_time <= date_to_split]

In [32]:
df_test = fraud_df[fraud_df.signup_time >= date_to_split]

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [70]:
X_train2 = X_train.loc[:, ['purchase_value', 'sex', 'age', 'Ads', 'Direct', 'SEO', 'time_to_purchase', 'Chrome',
       'FireFox', 'IE', 'Opera', 'Safari']] #dropped potential fraud due to data leakage concerns

In [71]:
y_train2 = y_train.loc[:, ['class']]

In [72]:
X_test2 = X_test.loc[:, ['purchase_value', 'sex', 'age', 'Ads', 'Direct', 'SEO', 'time_to_purchase', 'Chrome',
       'FireFox', 'IE', 'Opera', 'Safari']] #dropped potential fraud due to data leakage concerns

In [73]:
y_test2 = y_test.loc[:, ['class']]

In [54]:
rf_classifier = RandomForestClassifier(max_depth=10, n_estimators=1000)

In [57]:
X_train.head()

,purchase_value,sex,age,Ads,Direct,SEO,time_to_purchase,Chrome,FireFox,IE,Opera,Safari,possible_fraud
74068,26,0,41,1,0,0,0.000012,1,0,0,0,0,0
129509,19,1,34,0,0,1,103.275370,1,0,0,0,0,0
95250,12,0,40,1,0,0,11.437755,0,0,1,0,0,0
138881,16,1,34,0,0,1,60.341470,0,0,0,0,1,0
43829,42,1,22,0,0,1,94.494167,1,0,0,0,0,0


In [56]:
rf_classifier.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [58]:
preds = rf_classifier.predict_proba(X_test)

In [59]:
pos_preds = preds[:,1]

In [60]:
y_test.sum()

class    3544
dtype: int64

In [61]:
y_test.sum()/len(y_test)

class    0.093811
dtype: float64

In [62]:
rf_classifier.score(X_test, y_test)  #accuracy

0.9571178993064746

In [63]:
log_loss(y_test, pos_preds)

0.1771880982224652

In [74]:
rf_classifier.fit(X_train2, y_train2)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [75]:
preds2 = rf_classifier.predict_proba(X_test2)

In [76]:
rf_classifier.score(X_test2, y_test2)

0.9571178993064746

In [77]:
log_loss(y_test2, preds2)

0.1763394980657516